## Tests for the implementation of the multi-$Q^2$ kernel evolution

The return state of the dglap evolution should be a matrix not only in $x$ and flavour (as it is the case at the moment) but also on $Q^2$.

The goals are:

1. Minimize the amount of numba compilation. i.e., make sure we don't repeat calculations.
2. Parallelize the computation of the different values of $Q^2$ on the CPU.

Extra for the future:
- Compile the numba code into a GPU so the grid of $Q^2$ is computed on a GPU.

In [1]:
%load_ext autoreload
%autoreload 2
import logging
import numpy as np
import json
import eko.dglap as dglap
import eko.interpolation as interpolation
save_json = False

In [2]:
# Compute the input parameters
xgrid_low = interpolation.get_xgrid_linear_at_log(35,1e-7,0.1)
xgrid_mid = interpolation.get_xgrid_linear_at_id(15,0.1,0.95)
xgrid_high = 1.0-interpolation.get_xgrid_linear_at_log(5,1e-3,1.0 - 0.95)
xgrid = np.unique(np.concatenate((xgrid_low,xgrid_mid,xgrid_high)))
toy_xgrid = np.array([1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,.1,.3,.5,.7,.9])
polynom_rank = 4
Qref = np.sqrt(2)
Q0 = np.sqrt(2)
nf = 4

In [3]:
%%time
if save_json:
    ret1 = dglap.run_dglap({
        "PTO": 0,
        'alphas': 0.35,
        'Qref': Qref,
        'Q0': Q0,
        'NfFF': nf,

        "xgrid_type": "custom",
        "xgrid": xgrid,
        "xgrid_polynom_rank": polynom_rank,
        "log_interpol": True,
        "targetgrid": toy_xgrid,
        "Q2grid": [1e4]
    })

CPU times: user 3min 22s, sys: 262 ms, total: 3min 22s
Wall time: 3min 22s


In [4]:
#Save/load the operator to/from a json file
if save_json:
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)
    f = open('operators.json', 'w')
    json.dump(ret1["operators"], f, cls = NumpyEncoder)
    f.close()
else:
    f = open('operators.json', 'r')
    ret_backup = json.load(f)
    f.close()

In this first try we join the compute_singlet and compute_nonsinglet functions

In [34]:
%%time
if True:
    ret2 = dglap.run_dglap({
        "PTO": 0,
        'alphas': 0.35,
        'Qref': Qref,
        'Q0': Q0,
        'NfFF': nf,

        "xgrid_type": "custom",
        "xgrid": xgrid,
        "xgrid_polynom_rank": polynom_rank,
        "log_interpol": True,
        "targetgrid": toy_xgrid,
        "Q2grid": [1e4, 1e3]
    })

CPU times: user 4min 26s, sys: 160 ms, total: 4min 26s
Wall time: 4min 26s


In [37]:
for key, items in ret1["operators"].items():
    ir2 = ret2["operators"][key][:,:,0]
    np.testing.assert_allclose(items, ir2)

(11, 53, 2)